In [197]:
from dask import distributed
import pandas as pd

In [184]:
%%time
cluster = distributed.LocalCluster(n_workers=8)
client = distributed.Client(cluster)

/Users/nkthanh/opt/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


CPU times: user 103 ms, sys: 28.5 ms, total: 132 ms
Wall time: 228 ms


In [185]:
client

Client Scheduler: tcp://127.0.0.1:53341 Dashboard: http://127.0.0.1:53342/status,Cluster Workers: 8 Cores: 8 Memory: 8.59 GB


In [250]:
%%time
%run common.py

product_ids = get_product_id({'category_id': 1069, 'is_bundle': True})
# product_ids = get_product_id({'category_id': 15})
chunks = break_arr(product_ids, chunk_size=200)

print(f'Number of products {len(product_ids)}')
print(f'Number of chunks {len(chunks)}')

futures = []
for chunk in chunks:
    future = client.submit(load_products, chunk)
    futures.append(future)
    
df = pd.concat([pd.DataFrame(result) for result in client.gather(futures)])
df.shape

SELECT id FROM products WHERE category_id = %s AND is_bundle = %s LIMIT 1000
Number of products 300
Number of chunks 3
CPU times: user 49.3 ms, sys: 8.54 ms, total: 57.9 ms
Wall time: 146 ms


(300, 26)

In [251]:
df.head()

,Type,Name,Seller SKU,Category,Brand,Color,Model,Part number,Barcode,Unit,...,Width,Height,Short description,Description,Display name,meta keyword,Meta Description,Sale category,Tax in,Tax out
0,Bundle,S 0609 1k 2,None,Bé ngủ,1More,Nâu,M124,P13,None,Chiếc,...,17.0,17.0,Mô tả ngắn,None,SEO name,Meta Keywords,Meta Description,Miếng lót sơ sinh,None,None
1,Bundle,S 0609 1k 5,None,Bé ngủ,1More,Nâu,M127,P16,None,Chiếc,...,17.0,17.0,Mô tả ngắn,None,SEO name,Meta Keywords,Meta Description,Miếng lót sơ sinh,None,None
2,Bundle,S 0609 1k 8,None,Bé ngủ,1More,Nâu,M130,P19,None,Chiếc,...,17.0,17.0,Mô tả ngắn,None,SEO name,Meta Keywords,Meta Description,Miếng lót sơ sinh,None,None
3,Bundle,S 0609 1k 12,None,Bé ngủ,1More,Nâu,M124,P13,None,Chiếc,...,17.0,17.0,Mô tả ngắn,None,SEO name,Meta Keywords,Meta Description,Miếng lót sơ sinh,None,None
4,Bundle,S 0609 1k 15,None,Bé ngủ,1More,Nâu,M127,P16,None,Chiếc,...,17.0,17.0,Mô tả ngắn,None,SEO name,Meta Keywords,Meta Description,Miếng lót sơ sinh,None,None
